In [42]:
import matplotlib.pyplot as plt
import numpy as np
from math import floor, ceil
from cmath import phase
import matplotlib.animation as animation
from matplotlib.animation import PillowWriter

from shapely.geometry import LineString
from shapely.geometry import Point
import random as rd

In [2]:
def get_next(x, fwd=True):
    if x==int(x):
        return x+1 if fwd else x-1
    else:
        return floor(x)+1 if fwd else ceil(x)-1

def closest_point(arr, px):
    dist = list(map(lambda p: abs(p.x-px), arr))
    return arr[dist.index(min(dist))]
    
def circle_line_inter(cx, cy, r, px, py, npx, npy):
    p = Point(cx,cy)
    c = p.buffer(r).boundary
    l = LineString([(px,py), (npx, npy)])
    i = c.intersection(l)
    if i.type == "MultiPoint":
        i = closest_point(i, px)
    elif i.type != "Point":
        return None
    dir = complex(npx-px, npy-py)
    dir /= abs(dir)
    mid_dir = complex(i.x-cx, i.y-cy)
    mid_dir /= abs(mid_dir)

    scal = mid_dir.real*dir.real + mid_dir.imag*dir.imag

    new_dir = -2*scal*mid_dir+dir
    
    return (i.x, i.y, new_dir)



In [122]:
class CircleMirrors:
    def __init__(self, x,y,dx,dy,width=3, height=8, r=0.4)->None:
        self.width = width #number of columns
        self.height = 2*(height//2) #number of rows
        self.r = r #radius of the mirrors
        self.mirrors = []
        self.init_mirrors()
        self.new_ray(x,y,dx,dy)
    
    def new_ray(self, x,y,dx,dy):
        self.ray_px = [x]
        self.ray_py = [y]
        self.ray_dir = complex(dx,dy)
        self.ray_dir /= abs(self.ray_dir)
    
    
    def init_mirrors(self):
        for i in range(self.width):
            if (i%2)==0:
                self.mirrors.append([2*j for j in range(self.height//2+1)])
            else:
                self.mirrors.append([2*j+1 for j in range(self.height//2)])

    def get_circs(self):
        circs=[]
        for i in range(self.width):
            for j in self.mirrors[i]:
                if self.in_bound(i,j):
                    circs.append(plt.Circle((i,j), self.r, color='k'))
        return circs

    def is_ray_heading_out(self):
        if self.ray_px[-1]>=self.width+1 and self.ray_dir.real>0:
            # print("out1")
            return True
        if self.ray_px[-1]<=-1 and self.ray_dir.real<0:
            # print("out2")
            return True
        if self.ray_py[-1]>=self.height+1 and self.ray_dir.imag>0:
            # print("out3")
            return True
        if self.ray_py[-1]<=-1 and self.ray_dir.imag<0:
            # print("out4")
            return True
        return False
    def in_bound(self,x,y):
        return (x>=0 and x<= self.width-1 and y>=0 and y<=self.height and (x!=self.width//2 or y!=self.height//2))

    def check_for_inter(self, npx, npy, verbose=False):
        """npx is integer"""
        px, py = self.ray_px[-1], self.ray_py[-1]
        px += self.ray_dir.real/1000000 # to avoid double intersections
        py += self.ray_dir.imag/1000000

        up_dir = 1 if self.ray_dir.imag>=0 else -1
        left_dir = 1 if self.ray_dir.real>=0 else -1

        first_left_c = floor(py) if up_dir==1 else ceil(py)
        first_right_c = first_left_c+1 if up_dir==1 else first_left_c-1

        if (first_right_c%2)!= (npx+(left_dir-1)//2)%2:
            first_left_c -= 1*up_dir
            first_right_c -= 1*up_dir
        
        if verbose:
            print("inter_check")

        while (up_dir==1 and (first_left_c<=npy+1 or first_right_c<=npy+1)) or (up_dir==-1 and (first_left_c>=npy-1 or first_right_c>=npy-1)):
            x1 = npx-1 if left_dir==1 else npx+1
            y1 = first_left_c  if left_dir==1 else first_right_c

            x2 = npx if left_dir==1 else npx
            y2 = first_right_c  if left_dir==1 else first_left_c
            if verbose:
                print(x1,y1)
                print(x2,y2)
            if self.in_bound(x1,y1):
                c2 = circle_line_inter(x1,y1,self.r, px,py, npx,npy)
                if c2!=None:
                    return c2
            if self.in_bound(x2,y2):
                c1 = circle_line_inter(x2,y2, self.r, px, py, npx, npy)
                if c1!=None:
                    return c1
            
           
            first_left_c += 2*up_dir
            first_right_c += 2*up_dir
        if verbose:
            print("no inter")   

                    
                    

    def get_next_ray_inter(self, verbose=False):
        if self.is_ray_heading_out():
            # print("out!")
            return True
        fwd = True if self.ray_dir.real>0 else False

        # compute next point
        npx = get_next(self.ray_px[-1], fwd)
        ratio = (npx-self.ray_px[-1])/self.ray_dir.real
        npy = self.ray_py[-1]+self.ray_dir.imag*ratio


        potential_inter =  self.check_for_inter(npx,npy,verbose)
        if potential_inter != None:
            self.ray_px.append(potential_inter[0])
            self.ray_py.append(potential_inter[1])
            self.ray_dir = potential_inter[2]
        else:
            self.ray_px.append(npx)
            self.ray_py.append(npy)
            self.get_next_ray_inter(verbose)
    
        return False
    def draw(self):
       
        fig, ax = plt.subplots() # note we must use plt.subplots, not plt.subplot
        ax.set_xlim((-1, self.width))
        ax.set_ylim((-1, self.height+1))

        # draw circs
        for circ in self.get_circs():
            ax.add_patch(circ)

        # draw ray:
        ax.set_aspect('equal')
        ax.axis('off')

        ax.plot(self.ray_px, self.ray_py, '-r')


In [5]:
def draw_mutiple_rays(cm, rays):
    
    fig, ax = plt.subplots() # note we must use plt.subplots, not plt.subplot
    ax.set_xlim((-1, cm.width))
    ax.set_ylim((-1, cm.height+1))

    # draw ray:
    ax.set_aspect('equal')
    ax.axis('off')
    for ray in rays:
        ax.plot(ray[0], ray[1], color=(1.0,1-ray[2],ray[2]), linewidth=0.1)

In [6]:
%matplotlib qt

cm = CircleMirrors(5,6,1,0.1,10,10)

n=10
rays = []

for i in range(n):
    print(i)
    cm.new_ray(5,6,np.cos(2*np.pi*i/n+0.1),np.sin(2*np.pi*i/n+0.1))
    while(not cm.get_next_ray_inter()):
        pass
    rays.append([cm.ray_px, cm.ray_py,i/n])


0
1
2
3
4
5
6
7
8
9


In [120]:
L = 8
CM = CircleMirrors(5,6,1,0.1,L+1,L, 0.3)

fig, ax = plt.subplots() # note we must use plt.subplots, not plt.subplot
ax.set_xlim((-2, CM.width+1))
ax.set_ylim((-2, CM.height+2))
ax.set_aspect('equal')
ax.axis('off')
for circ in CM.get_circs():
    ax.add_patch(circ)
ax.plot([L/2],[L/2],'or')

line, = plt.plot([],[], '-r') 

# fonction à définir quand blit=True
# crée l'arrière de l'animation qui sera présent sur chaque image
def init():
    line.set_data([],[])
    return line,

def onmove(event):
    try:
        p = complex(L/2+0.5,L/2+0.5)
        z = complex(event.xdata, event.ydata)-p
        z /= abs(z)
        CM.new_ray(p.real,p.imag, z.real, z.imag)
    except:
        pass
    # print('%s click: button=%d, x=%d, y=%d, xdata=%f, ydata=%f' %
    #       ('double' if event.dblclick else 'single', event.button,
    #        event.x, event.y, event.xdata, event.ydata))

def onclick(event):
    print(cm.ray_px[0],cm.ray_py[0], cm.ray_dir.real, cm.ray_dir.imag)
# cid = fig.canvas.mpl_connect('motion_notify_event', onmove)
# cid = fig.canvas.mpl_connect('button_press_event', onclick)


N = 1000
def animate(i): 
    phase = -2*np.pi*i/N/10000 + 1.3
    CM.new_ray(L/2,L/2, np.cos(phase),np.sin(phase))
    while(not CM.get_next_ray_inter()):
        pass
    line.set_data(CM.ray_px,CM.ray_py)
    return line,
 
ani = animation.FuncAnimation(fig, animate, frames=N, init_func=init, blit=True, interval=10, repeat=False)



In [121]:


ani.save('lasers.gif',   writer=PillowWriter(fps=24))

In [8]:
# shooting ray:

ray_px = [0,1]
ray_py = [0,1]

def dist(x1,x2,y1,y2):
    dx = x1-x2
    dy = y1-y2
    return np.sqrt(dx*dx+dy*dy)

def fetch_next(x, y, ray_px, ray_py, i, delta):
    if i == len(ray_px):
        return None
    d = dist(x, ray_px[i], y, ray_py[i])
    
    if d >= delta:
        ratio = delta/d       
        return i, x + ratio*(ray_px[i]-x),  y+ratio*(ray_py[i]-y)
    else:
        return fetch_next(ray_px[i], ray_py[i], ray_px, ray_py, i+1, delta-d)

def shoot_from_still(ray_px, ray_py, delta=0.2):
    i = 1
    x = ray_px[0]
    y = ray_py[0]
    new_x=[]
    new_y=[]
    res = fetch_next(x,y,ray_px,ray_py, i, delta)
    while (not(res is None)):
        new_x.append(x)
        new_y.append(y)
        i, x, y = res
        res = fetch_next(x, y, ray_px, ray_py, i, delta)

    return new_x, new_y



new_x, new_y = shoot_from_still(cm.ray_px,cm.ray_py)
len(new_x)

205

In [114]:
global Line_number, Lg, S,F,G
Line_number=50
Lg = 11
S = [0 for i in range(Line_number)]
F = [1 for i in range(Line_number)]
G = [10 for i in range(Line_number)]




# Rays and mirros:
CM = [CircleMirrors(5,6,1,0.1,Lg+1,Lg, 0.3) for i in range(Line_number)]
def init_news(cm):
    global Lg,S,F,G
    cm.new_ray(Lg/2,Lg/2, 0.5-rd.random(), 0.5-rd.random())
    while(not cm.get_next_ray_inter()):
        pass
    return shoot_from_still(cm.ray_px,cm.ray_py)
new_x=[]
new_y=[]
for i in range(Line_number):
    nx,ny = init_news(CM[i])
    new_x.append(nx)
    new_y.append(ny)

# fig and circles
# fig, ax = plt.subplots() # note we must use plt.subplots, not plt.subplot
# ax.set_xlim((-2, CM[0].width+1))
# ax.set_ylim((-2, CM[0].height+2))
# ax.set_aspect('equal')
# ax.axis('off')
# for circ in CM[0].get_circs():
#     ax.add_patch(circ)

# # lines
# lines = [ax.plot([],[], '-r', linewidth=2)[0] for i in range(Line_number)]
# def init():
#     for line in lines:
#         line.set_data([],[])
#     return lines

# def animate(i): 
#     print(i)
#     global Lg,S,F,G,Line_number, new_x, new_y   
#     for i in range(Line_number):
#         if (F[i]-S[i])>=G[i]:
#             S[i] += 1
#         F[i] += 1
#         if S[i]>=len(new_x[i]):
#             pass
#             # new_x[i], new_y[i] = init_news(CM[i])
#             # S[i] = 0
#             # F[i] = 1
#         lines[i].set_data(new_x[i][S[i]:F[i]], new_y[i][S[i]:F[i]])
#     return lines
 
# ani = animation.FuncAnimation(fig, animate, init_func=init, frames = 400,blit=True, interval=10, repeat=False)



In [115]:
ani.save('bouncing.gif',   writer=PillowWriter(fps=24))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [95]:
global Lg, S,F,G
Line_number=100
Lg = 8
S = [0]
F = [1]
G = [10]




delta=0.1



# Rays and mirros:
def new_mirror():
    return CircleMirrors(5,6,1,0.1,Lg+1,Lg, 0.4)
CM = new_mirror()

def init_news(cm):
    global Lg,S,F,G
    cm.new_ray(Lg/2,Lg/2, (rd.random()-0.5), (rd.random()-0.5))
    while(not cm.get_next_ray_inter()):
        pass
    return shoot_from_still(cm.ray_px,cm.ray_py,delta)

nx,ny = init_news(CM)
new_x=[nx]
new_y=[ny]


# fig and circles
fig, ax = plt.subplots() # note we must use plt.subplots, not plt.subplot

ax.set_xlim((-2, CM.width+1))
ax.set_ylim((-2, CM.height+2))
ax.set_aspect('equal')
ax.axis('off')

# for circ in CM[0].get_circs():
#     ax.add_patch(circ)

# lines
def new_line():
    return ax.plot([],[], '-r',  alpha=0.3)[0]

lines = [new_line()]
def init():
    for line in lines:
        line.set_data([],[])
    return lines

def animate(i): 
    global Lg,S,F,G, new_x, new_y 
    if F[-1]<len(new_x[-1]):
        F[-1] += 100
        lines[-1].set_data(new_x[-1][S[-1]:F[-1]], new_y[-1][S[-1]:F[-1]])
    
    else:
        lines.append(new_line())
        CM=new_mirror()
        S=[0]
        F=[1]
        new_x[-1], new_y[-1] = init_news(CM)
    
    return lines
 
ani = animation.FuncAnimation(fig, animate, init_func=init, blit=True, interval=10, repeat=False)


In [91]:
CM = CircleMirrors(5,6,1,0.1,Lg+1,Lg, 0.3)

# fig, ax = plt.subplots() # note we must use plt.subplots, not plt.subplot
# ax.set_xlim((-1, CM.width))
# ax.set_ylim((-1, CM.height+1))
# ax.set_aspect('equal')
# ax.axis('off')
# for circ in CM.get_circs():
#     ax.add_patch(circ)
# ax.plot([L/2],[L/2],'or')

N = 100000
s_angles = []
angles = []
for i in range(N):
    phase = 2*np.pi*i/N +0.01
    CM.new_ray(L/2,L/2, np.cos(phase),np.sin(phase))
    while(not CM.get_next_ray_inter()):
        pass
    # ax.plot(CM.ray_px,CM.ray_py, '-r', lw=0.1)
    s_angles.append(phase)
    angles.append(np.angle(complex(CM.ray_px[-1]-CM.ray_px[-2],CM.ray_py[-1]-CM.ray_py[-2])))


In [94]:
plt.scatter(s_angles, angles, s=0.3)
plt.xlabel("Input angle (N=100,000)")
plt.ylabel("Output angle")

Text(0, 0.5, 'Output angle')